In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np
# Write any helper functions you need here
def cost_hamiltonian(edges):
    """
    This function build the QAOA cost Hamiltonian for a graph, encoded in its edges

    Args:
    - Edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Operator: The cost Hamiltonian associated with the graph.
    """

    # Put your code here #
    terms = []
    coeffs = []

    vertices = set()
    
    # 遍历每条边
    for edge in edges:
        i, j = edge
        vertices.update(edge)  # 收集顶点

        # 添加 Z_i Z_j 项
        terms.append(qml.PauliZ(i) @ qml.PauliZ(j))
        coeffs.append(3/4)

        # 添加 Z_i 和 Z_j 项
        terms.append(qml.PauliZ(i))
        coeffs.append(3/4)
        
        terms.append(qml.PauliZ(j))
        coeffs.append(3/4)

    # 添加顶点的单独 Z_i 项 (即 -Z_i)
    for vertex in vertices:
        terms.append(qml.PauliZ(vertex))
        coeffs.append(-1)

    # 构建哈密顿量
    return qml.Hamiltonian(coeffs, terms)
def mixer_hamiltonian(edges):
    """
    This function build the QAOA mixer Hamiltonian for a graph, encoded in its edges

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Operator: The mixer Hamiltonian associated with the graph.
    """

    # Put your code here #
    terms = []
    num_vertices = set(vertex for edge in edges for vertex in edge)
    for vertex in num_vertices:
        terms.append(qml.PauliX(vertex))
    
    return qml.Hamiltonian(coeffs=[1] * len(terms), observables=terms)
def qaoa_circuit(params, edges):
    """
    This quantum function (i.e. a list of gates describing a circuit) implements the QAOA algorithm
    You should only include the list of gates, and not return anything

    Args:
    - params (np.array): A list encoding the QAOA parameters. You are free to choose any array shape you find 
    convenient.
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - This function does not return anything. Do not write any return statements.
    
    """

    # Put your code here
    vertices = set()
    for edge in edges:
        vertices.update(edge)
    num_vertices = len(vertices)

    # 1. 初始化量子态，应用 Hadamard 门
    for vertex in range(num_vertices):
        qml.Hadamard(wires=vertex)

    # 2. 计算 QAOA 层数
    p = len(params) // 2  # 每层有两个参数：gamma 和 beta

    # 3. 应用 QAOA 层
    for i in range(p):
        gamma = params[2 * i]   # 成本哈密顿量参数
        beta = params[2 * i + 1]  # 混频器哈密顿量参数

        # 4. 应用成本哈密顿量（Cost Hamiltonian）
        for edge in edges:
            # # 对边 (i, j) 应用 CZ 门 (控制-Z 门)
            # qml.CZ(wires=(edge[0], edge[1]))
            # # 分别对顶点 i 和 j 应用 RZ 旋转 (根据 Z_i Z_j 项以及 Z_i, Z_j 项)
            # qml.RZ(2 * gamma * 3/4, wires=edge[0])   # 乘以 3/4 系数
            # qml.RZ(2 * gamma * 3/4, wires=edge[1])   # 乘以 3/4 系数
            qml.CNOT(wires=(edge[0], edge[1]))
            qml.RZ(2 * gamma, wires=edge[1])
            qml.CNOT(wires=(edge[0], edge[1]))

        # 5. 每个顶点的单独 Z_i 项 (额外的 -Z_i 项)
        for vertex in vertices:
            qml.RZ(2 * gamma * (-1), wires=vertex)  # 对每个顶点再施加额外的 Z_i 项

        # 6. 应用混频器哈密顿量 (Mixer Hamiltonian)
        for vertex in range(num_vertices):
            qml.RX(2 * beta, wires=vertex)  # 对每个顶点应用 RX 旋转
# This function runs the QAOA circuit and returns the expectation value of the cost Hamiltonian

dev = qml.device("default.qubit")

@qml.qnode(dev)
def qaoa_expval(params, edges):
    qaoa_circuit(params, edges)
    return qml.expval(cost_hamiltonian(edges))

def optimize(edges):
    """
    This function returns the parameters that minimize the expectation value of
    the cost Hamiltonian after applying QAOA

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    
    """

    # Your cost function should be the expectation value of the cost Hamiltonian
    # You may use the qaoa_expval QNode defined above
    
    # Write your optimization routine here
    p = 2 * len(edges)  # QAOA 层数
    params = np.random.uniform(-1, 1, 2 * p)  # 随机初始化参数

    # 使用 PennyLane 的 Adam 优化器
    # opt = qml.AdamOptimizer(stepsize=step_size)
    steps=1
    for i in range(2):
        steps = steps * 0.1
        opt = qml.NesterovMomentumOptimizer(stepsize=steps)

        # 优化参数
        for step in range(200):
            # 通过最小化期望值来更新参数
            params = opt.step(lambda v: qaoa_expval(v, edges), params)

            # 每 10 步打印一次结果
            if step % 100 == 0:
                cost = qaoa_expval(params, edges)
                # print(f"Step {i*300+step}: Cost = {cost}")

    return params 

# These are auxiliary functions that will help us grade your solution. Feel free to check them out!

@qml.qnode(dev)
def qaoa_probs(params, edges):
  qaoa_circuit(params, edges)
  return qml.probs()

def approximation_ratio(params, edges):

    true_min = np.min(qml.eigvals(cost_hamiltonian(edges)))

    approx_ratio = qaoa_expval(params, edges)/true_min

    return approx_ratio

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = optimize(ins)
    output= approximation_ratio(params,ins).numpy()

    ground_energy = np.min(qml.eigvals(cost_hamiltonian(ins)))

    index = np.argmax(qaoa_probs(params, ins))
    vector = np.zeros(len(qml.matrix(cost_hamiltonian(ins))))
    vector[index] = 1

    calculate_energy = np.real_if_close(np.dot(np.dot(qml.matrix(cost_hamiltonian(ins)), vector), vector))
    verify = np.isclose(calculate_energy, ground_energy)

    if verify:
      return str(output)
    
    return "QAOA failed to find right answer"

def check(solution_output: str, expected_output: str) -> None:

    assert not solution_output == "QAOA failed to find right answer", "QAOA failed to find the ground eigenstate."
        
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert solution_output >= expected_output-0.01, "Minimum approximation ratio not reached"

# These are the public test cases
test_cases = [
    ('[[0, 1], [1, 2], [0, 2], [2, 3]]', '0.55'),
    ('[[0, 1], [1, 2], [2, 3], [3, 0]]', '0.92'),
    ('[[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4]]', '0.55')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0, 1], [1, 2], [0, 2], [2, 3]]'...
Runtime Error. Input is not an Operator, tape, QNode, or quantum function
Running test case 1 with input '[[0, 1], [1, 2], [2, 3], [3, 0]]'...
Runtime Error. Input is not an Operator, tape, QNode, or quantum function
Running test case 2 with input '[[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4]]'...
Runtime Error. Input is not an Operator, tape, QNode, or quantum function


In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

In [3]:
def cost_hamiltonian(edges):
    """
    This function builds the QAOA cost Hamiltonian for a graph, encoded in its edges.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Operator: The cost Hamiltonian associated with the graph.
    """
    
    # 输入检查
    if not isinstance(edges, list):
        raise ValueError("Edges must be a list of edges.")
    
    for edge in edges:
        if not (isinstance(edge, list) and len(edge) == 2 and all(isinstance(v, int) for v in edge)):
            raise ValueError("Each edge must be a list of two integers.")
    
    terms = []
    
    # 遍历每条边
    for edge in edges:
        i, j = edge
        # 添加 Z_ij 项
        terms.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Z_i Z_j
        terms.append(qml.PauliZ(i))                    # Z_i
        terms.append(qml.PauliZ(j))                    # Z_j

    # 返回哈密顿量
    return (3/4) * qml.Hamiltonian(coefs=[1] * len(terms), obs=terms)


In [4]:
def mixer_hamiltonian(edges):
    """
    This function builds the QAOA mixer Hamiltonian for a graph, encoded in its edges.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Operator: The mixer Hamiltonian associated with the graph.
    """

    # 输入检查
    if not isinstance(edges, list):
        raise ValueError("Edges must be a list of edges.")
    
    # 获取所有的顶点
    vertices = set()
    for edge in edges:
        if len(edge) != 2:
            raise ValueError("Each edge must be a list of two integers.")
        vertices.update(edge)

    # 创建混频器哈密顿量
    terms = []
    for vertex in vertices:
        terms.append(qml.PauliX(vertex))  # 添加 X_i 项

    # 返回混频器哈密顿量
    return qml.Hamiltonian(coefs=[1] * len(terms), obs=terms)


In [5]:
def qaoa_circuit(params, edges):
    """
    This quantum function implements the QAOA circuit.

    Args:
    - params (np.array): A list encoding the QAOA parameters.
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.
    """

    # 获取顶点集合
    vertices = set()
    for edge in edges:
        vertices.update(edge)
    num_vertices = len(vertices)

    # 应用 Hadamard 门
    for vertex in range(num_vertices):
        qml.Hadamard(wires=vertex)

    # QAOA层数
    p = len(params) // 2  # 假设每层有一个 gamma 和一个 beta
    for i in range(p):
        gamma = params[2 * i]   # 成本哈密顿量参数
        beta = params[2 * i + 1]  # 混频器哈密顿量参数

        # 应用成本哈密顿量的相位旋转
        for edge in edges:
            qml.CZ(wires=(edge[0], edge[1]))  # 应用 CZ 门
            qml.RZ(2 * gamma, wires=edge[0])   # 旋转
            qml.RZ(2 * gamma, wires=edge[1])   # 旋转

        # 应用混频器哈密顿量的旋转
        for vertex in range(num_vertices):
            qml.RX(2 * beta, wires=vertex)


In [6]:
def optimize(edges):
    """
    This function returns the parameters that minimize the expectation value of
    the cost Hamiltonian after applying QAOA.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - np.array: The optimal parameters for QAOA.
    """
    # 初始化参数
    p = 1  # QAOA 的层数
    params = np.random.rand(2 * p)  # 随机初始化参数

    # 定义成本函数
    def cost_function(params):
        return qaoa_expval(params, edges)

    # 使用 SciPy 的优化函数进行优化
    from scipy.optimize import minimize

    result = minimize(cost_function, params, method='BFGS')
    
    return result.x  # 返回最优参数


In [7]:
import json
import pennylane as qml
import pennylane.numpy as np
from scipy.optimize import minimize

# Cost Hamiltonian
def cost_hamiltonian(edges):
    """
    This function builds the QAOA cost Hamiltonian for a graph, encoded in its edges.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Hamiltonian: The cost Hamiltonian associated with the graph.
    """
    terms = []
    coeffs = []

    vertices = set()
    
    # 遍历每条边
    for edge in edges:
        i, j = edge
        vertices.update(edge)  # 收集顶点

        # 添加 Z_i Z_j 项
        terms.append(qml.PauliZ(i) @ qml.PauliZ(j))
        coeffs.append(3/4)

        # 添加 Z_i 和 Z_j 项
        terms.append(qml.PauliZ(i))
        coeffs.append(3/4)
        
        terms.append(qml.PauliZ(j))
        coeffs.append(3/4)

    # 添加顶点的单独 Z_i 项 (即 -Z_i)
    for vertex in vertices:
        terms.append(qml.PauliZ(vertex))
        coeffs.append(-1)

    # 构建哈密顿量
    return qml.Hamiltonian(coeffs, terms)


# Mixer Hamiltonian
def mixer_hamiltonian(edges):
    terms = []
    num_vertices = set(vertex for edge in edges for vertex in edge)
    for vertex in num_vertices:
        terms.append(qml.PauliX(vertex))
    
    return qml.Hamiltonian(coeffs=[1] * len(terms), observables=terms)

def qaoa_circuit(params, edges):
    """
    This quantum function implements the QAOA circuit for the given cost Hamiltonian.
    
    Args:
    - params (np.array): A list encoding the QAOA parameters.
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
      the edges of the graph for which we need to solve the minimum vertex cover problem.
    """

    # 获取图的所有顶点
    vertices = set()
    for edge in edges:
        vertices.update(edge)
    num_vertices = len(vertices)

    # 1. 初始化量子态，应用 Hadamard 门
    for vertex in range(num_vertices):
        qml.Hadamard(wires=vertex)

    # 2. 计算 QAOA 层数
    p = len(params) // 2  # 每层有两个参数：gamma 和 beta

    # 3. 应用 QAOA 层
    for i in range(p):
        gamma = params[2 * i]   # 成本哈密顿量参数
        beta = params[2 * i + 1]  # 混频器哈密顿量参数

        # 4. 应用成本哈密顿量（Cost Hamiltonian）
        for edge in edges:
            # # 对边 (i, j) 应用 CZ 门 (控制-Z 门)
            # qml.CZ(wires=(edge[0], edge[1]))
            # # 分别对顶点 i 和 j 应用 RZ 旋转 (根据 Z_i Z_j 项以及 Z_i, Z_j 项)
            # qml.RZ(2 * gamma * 3/4, wires=edge[0])   # 乘以 3/4 系数
            # qml.RZ(2 * gamma * 3/4, wires=edge[1])   # 乘以 3/4 系数
            qml.CNOT(wires=(edge[0], edge[1]))
            qml.RZ(2 * gamma, wires=edge[1])
            qml.CNOT(wires=(edge[0], edge[1]))

        # 5. 每个顶点的单独 Z_i 项 (额外的 -Z_i 项)
        for vertex in vertices:
            qml.RZ(2 * gamma * (-1), wires=vertex)  # 对每个顶点再施加额外的 Z_i 项

        # 6. 应用混频器哈密顿量 (Mixer Hamiltonian)
        for vertex in range(num_vertices):
            qml.RX(2 * beta, wires=vertex)  # 对每个顶点应用 RX 旋转



# Expectation Value
dev = qml.device("default.qubit")

@qml.qnode(dev)
def qaoa_expval(params, edges):
    qaoa_circuit(params, edges)
    return qml.expval(cost_hamiltonian(edges))

def optimize(edges):
    """
    Optimize the QAOA parameters using PennyLane's AdamOptimizer.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers representing the edges of the graph.
    - max_steps (int): The maximum number of optimization steps.
    - step_size (float): The step size for the optimizer.

    Returns:
    - np.array: The optimized parameters.
    """
    p = 2 * len(edges)  # QAOA 层数
    params = np.random.uniform(-1, 1, 2 * p)  # 随机初始化参数

    # 使用 PennyLane 的 Adam 优化器
    # opt = qml.AdamOptimizer(stepsize=step_size)
    steps=1
    for i in range(2):
        steps = steps * 0.1
        opt = qml.NesterovMomentumOptimizer(stepsize=steps)

        # 优化参数
        for step in range(200):
            # 通过最小化期望值来更新参数
            params = opt.step(lambda v: qaoa_expval(v, edges), params)

            # 每 10 步打印一次结果
            if step % 100 == 0:
                cost = qaoa_expval(params, edges)
                print(f"Step {i*300+step}: Cost = {cost}")

    # opt = qml.GradientDescentOptimizer(stepsize=0.00001)
    # # 优化参数
    # for step in range(2000):
    #     # 通过最小化期望值来更新参数
    #     params = opt.step(lambda v: qaoa_expval(v, edges), params)

    #     # 每 10 步打印一次结果
    #     if step % 100 == 0:
    #         cost = qaoa_expval(params, edges)
    #         print(f"Step {step}: Cost = {cost}")

    return params


# QAOA Probabilities
@qml.qnode(dev)
def qaoa_probs(params, edges):
    qaoa_circuit(params, edges)
    return qml.probs()

# Approximation Ratio
def approximation_ratio(params, edges):
    true_min = np.min(qml.eigvals(cost_hamiltonian(edges)))
    approx_ratio = qaoa_expval(params, edges) / true_min
    return approx_ratio

# Run Function
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    print(f"Edges: {ins}")

    # 调用优化函数
    params = optimize(ins)
    print(f"Optimized params: {params}")

    # 计算近似比率
    output = approximation_ratio(params, ins)
    print(f"Approximation ratio: {output}")

    # 计算真实最小能量
    ground_energy = np.min(qml.eigvals(cost_hamiltonian(ins)))
    print(f"Ground energy: {ground_energy}")

    # 计算基态的概率分布并寻找最高概率的解
    index = np.argmax(qaoa_probs(params, ins))
    vector = np.zeros(len(qml.matrix(cost_hamiltonian(ins))))
    vector[index] = 1

    # 计算 QAOA 电路输出状态的能量
    calculate_energy = np.real_if_close(np.dot(np.dot(qml.matrix(cost_hamiltonian(ins)), vector), vector))
    print(f"Calculated energy from QAOA: {calculate_energy}")

    # 检查计算出的能量是否接近真实最小能量
    verify = np.isclose(calculate_energy, ground_energy)
    print(f"Energy verification: {verify}")

    # 如果能量验证成功，则返回近似比率
    if verify:
        return str(output)
    
    return "QAOA failed to find the right answer"


# Check Function
def check(solution_output: str, expected_output: str) -> None:
    assert not solution_output == "QAOA failed to find the right answer", "QAOA failed to find the ground eigenstate."
        
    solution_output = float(solution_output)
    expected_output = float(expected_output)
    print(solution_output, expected_output)
    assert solution_output >= expected_output - 0.01, "Minimum approximation ratio not reached"

# Public Test Cases
test_cases = [
    ('[[0, 1], [1, 2], [0, 2], [2, 3]]', '0.55'),
    ('[[0, 1], [1, 2], [2, 3], [3, 0]]', '0.92'),
    ('[[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4]]', '0.55')
]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")
    try:
        output = run(input_)
    except Exception as exc:
        print(f"Runtime Error: {exc}")
    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")
        else:
            print("Correct!")


Running test case 0 with input '[[0, 1], [1, 2], [0, 2], [2, 3]]'...
Edges: [[0, 1], [1, 2], [0, 2], [2, 3]]
Step 0: Cost = 0.5009135379513776
Step 100: Cost = -0.30741984468756106
Step 300: Cost = -0.795293166266376
Step 400: Cost = -2.8727406269526714
Optimized params: [-23.26450642 -20.21226803   5.35220961  33.1092499  -22.89902599
  -7.33740793  -0.96999503  -5.5835301  -95.80283214  15.2762397
 -95.8549368  -34.78610353 -33.91265874   7.27653724 -61.90634729
  -1.87147428]
Approximation ratio: 0.9222704211954446
Ground energy: -3.0
Calculated energy from QAOA: -3.0
Energy verification: True
0.9222704211954446 0.55
Correct!
Running test case 1 with input '[[0, 1], [1, 2], [2, 3], [3, 0]]'...
Edges: [[0, 1], [1, 2], [2, 3], [3, 0]]
Step 0: Cost = 0.1461709875592599
Step 100: Cost = 0.7792885913920764
Step 300: Cost = -0.85617385830945
Step 400: Cost = -2.9811490310305744
Optimized params: [ 65.21125974  15.6993966   94.86992662  23.78201906  10.7942336
 -17.64797776  -8.10897104 -2

In [8]:
def cost_hamiltonian(edges):
    """
    This function builds the QAOA cost Hamiltonian for a graph, encoded in its edges.

    Args:
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.

    Returns:
    - pennylane.Hamiltonian: The cost Hamiltonian associated with the graph.
    """
    terms = []
    for edge in edges:
        print(f"Edge: {edge}")  # 调试输出
        term = qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])
        terms.append(term)
        print(f"Hamiltonian term for edge {edge}: {term}")
    
    # 使用 (3/4) 作为系数构造哈密顿量
    hamiltonian = qml.Hamiltonian(coeffs=[3/4] * len(terms), observables=terms)
    print(f"Constructed cost Hamiltonian: {hamiltonian}")
    return hamiltonian
qml.eigvals(cost_hamiltonian([[0, 1], [1, 2], [0, 2], [2, 3]]))

Edge: [0, 1]
Hamiltonian term for edge [0, 1]: Z(0) @ Z(1)
Edge: [1, 2]
Hamiltonian term for edge [1, 2]: Z(1) @ Z(2)
Edge: [0, 2]
Hamiltonian term for edge [0, 2]: Z(0) @ Z(2)
Edge: [2, 3]
Hamiltonian term for edge [2, 3]: Z(2) @ Z(3)
Constructed cost Hamiltonian: 0.75 * (Z(0) @ Z(1)) + 0.75 * (Z(1) @ Z(2)) + 0.75 * (Z(0) @ Z(2)) + 0.75 * (Z(2) @ Z(3))


array([-1.5, -1.5, -1.5, -1.5, -1.5, -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  1.5,  1.5,  3. ,  3. ])

In [9]:
edges = [[0, 1], [1, 2], [0, 2], [2, 3],[5,9]]
params = np.random.rand(2*5)

In [10]:
# QAOA Circuit
def qaoa_circuit(params, edges):
    """
    This quantum function implements the QAOA circuit.

    Args:
    - params (np.array): A list encoding the QAOA parameters.
    - edges (list(list(int))): A list of ordered pairs of integers, representing 
    the edges of the graph for which we need to solve the minimum vertex cover problem.
    """

    # Get all vertices in the graph
    vertices = set()
    for edge in edges:
        vertices.update(edge)
    num_vertices = len(vertices)

    # 1. Apply Hadamard gates to initialize each qubit in an equal superposition state
    for vertex in range(num_vertices):
        qml.Hadamard(wires=vertex)

    # 2. Number of QAOA layers
    p = len(params) // 2  # Assume each layer has four parameters: gamma1, gamma2, beta1, beta2

    # 3. Apply QAOA layers
    for i in range(p):
        gamma = params[2 * i]     # Parameter for the first RZ rotation
        beta = params[2 * i + 1]  # Parameter for the first RX rotation
  

        # 4. Apply cost Hamiltonian phase rotations
        for edge in edges:
            qml.CNOT(wires=(edge[0], edge[1]))  # Control-Z gate for interaction
            qml.RZ(2 * gamma, wires=edge[1])  # RZ rotation for vertex i
            qml.CNOT(wires=(edge[0], edge[1]))  # Control-Z gate for interaction
            # qml.RZ(2 * gamma2, wires=edge[1])  # RZ rotation for vertex j

        # 5. Apply mixer Hamiltonian rotations
        for vertex in range(num_vertices):
            qml.RX(2 * beta, wires=vertex)  # RX rotation for each vertex
qaoa_circuit(params, edges)

In [14]:
minimize(cost_function, params, method='BFGS')

NameError: name 'cost_function' is not defined